# DATA 422 - Group Project

## Data Collection

For this project we were interested in football statisitcs of the top 7 teams from both the Premier League and La Liga. To ensure our visualisations did not contiain too much information (to the point where they break down), we only looked at years dating back to 2015. However, many of the functions below are capable of retrieving data from much futher back than this.<br><br>
This notebook contains all the functions and programs used in scraping our football data. The majority of the data was retrieved from www.fbref.com - this was a one stop shop for all football statistics across a multitude of leagues. This was used to collect team statistics, end of year tables and wage data. Additionally, www.statbunker.com was used to scrape data detailing individual player statistics - including fantasy points. <br> <br>
In total 7 dataframes were scraped from these websites. A brief summary of each is listed below: <br>
* Summary - This contains a brief summary of each leagues. Included is the championship winning team and the top goal scorer. 
* TablesPL - This is the end of year tables for the Premier League competition. Aside from each teams final ranking, it also contains a variety of statisitcs detailing how well each team performed. 
* TablesLL - This contains the end of year tables for the La Liga competition. 
* FantasyPL - This contains detailed statistics for each player, in the Premier League, for a given season (including their final fantasy points).
* FantasyLL - This is the equivalent for the La Liga competition.
* WagesPL - This contains wages paid by teams in the Premier League.
* WagesLL - This is the La Liga equivalent.

### Libraries Used

In [1]:
#Libraries Used
library(magrittr) 
library(purrr) 
library(glue)
library(stringr) 
library(rvest) 
library(polite) 
library(xml2)
library(dplyr)
library(tidyr)
library(tidyverse)
library(skimr)


Attaching package: 'purrr'


The following object is masked from 'package:magrittr':

    set_names



Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



Attaching package: 'tidyr'


The following object is masked from 'package:magrittr':

    extract


── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ readr     2.1.4
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     
── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::extract()        masks magrittr::extract()
✖ dplyr::filter()         masks stats::filter()
✖ readr::guess_encoding() masks rvest::guess_encoding()
✖ dplyr::lag()            masks stats::lag()
✖ purrr::set_names()      masks magrittr::set_nam

## Premier League/La Liga - Summary  

This returns the championship winning team and top goal scorer for both leagues from the last 8 years. With the additional league, season and ID columns; a 16 x 9  dataset is returned.
<br> 

**Source: www.fbref.com** 

### Functions 

A number of functions defined here are also used in subsequent programs

In [2]:
#Function to create list of years to search through
generate_years <- function(start, end){
                    years = seq(start, end)
    
    return(years)
}


#Function to aid in searching for correct year. E.g. converts 2023 to 2022-2023
search_season <- function(year){
                    minus_1 = as.character(year - 1)
                    season_search = paste0(minus_1, "-" , year)
                    
    return(season_search)
}


#Converts League into format of website
convert_league = function(league){
                    words = unlist(strsplit(league, " "))
                    conversion = glue("-", words[1], "-", words[2], "-Stats")
    
    return(conversion)    
}


#Gets correct number for link - dependent on which league we're looking at
comp_num = function(league){
            if(league == "Premier League"){
                num = "9/"
            }else{
                num = "12/"
            }
    
    return(num)
}


#Creates link for initial seach of each year 
season_html <- function(year, base_url, league){
                #Collecting pieces to create link
                season = search_season(year)
                num = comp_num(league)
                league_conversion = convert_league(league)
    
                #Creating link
                link = glue(base_url, num, season, "/", season, league_conversion)
    
                #Reading in link
                season_html <- read_html(link)
    
    return(season_html)
}


#Season champion
find_champs = function(html_doc){
                champ_team = html_doc %>%
                             html_nodes(xpath = "//*[@id='meta']/div[2]/p[4]/a") %>% #Selector
                             html_text()
            
    return(champ_team)
}


#Gather information on top goal scorer and goals scored
top_goals = function(html_doc){
                #Top goal scorer
                top_goal_scorer = html_doc %>%
                            html_nodes("strong:contains('Most Goals') + a") %>%
                            html_text()
                #Goals scored
                goals_scored = html_doc %>%
                            html_nodes(xpath = "//*[@id='meta']/div[2]/p[5]/span") %>%
                            html_text()
    
    return(c(top_goal_scorer, goals_scored))
    
}

#Function to return the correct competition ID - used in creating link
league_num = function(league){
                if(league == "Premier League"){
                    num = "91"
                }else{
                    num = "121"
                }
    return(num)
}
#Get number of wins by champ team
champs_wins = function(html_doc, year, league){
                season = search_season(year)
                num = league_num(league)
                node = glue("#results{season}{num}_overall > tbody > tr:nth-child(1) > td:nth-child(4)")
                wins = html_doc %>%        
                        html_nodes(node) %>%
                         html_text() 
    return(wins)
}  


#Little function to convert league into intials
league_intitials <- function(league){
                        if(league == "Premier League"){
                            initials = "PL"
                        } else{
                            initials = "LL"
                        }
    return(initials)
}


#Create and build summary dataframe
update_summary = function(year, league, champs, wins, top_goals){   
                    #Create dataframe for each season
                    summary_df = tibble(Season = search_season(year), 
                                        League = league,
                                        Champions = champs,
                                        Games_Won = as.numeric(wins),
                                        Top_Goal_Scorer = top_goals[1],
                                        Goals_Scored = as.numeric(top_goals[2]))
       
                    #Adding Season_Id column
                    short_year = paste0(substr(summary_df$Season, 3, 4), "/", substr(summary_df$Season, 8, 9)) #Shortening the season summary for a better ID
                    league_initials = league_intitials(league) #e.g Converts Premier League to PL
    
                    summary_df %<>%
                            mutate(Season_ID = as.character(glue("{short_year}-{league_initials}"))) 
    
                    #Update global summary dataframe
                    if(is.null(summary)){
                        summary <<- summary_df #If first iteration -> create summary table
                    } else {
                        summary <<- bind_rows(summary, summary_df) #If any other iteration -> add row
                    }
                    
    return(summary)
}



#Create ID Columns
create_ids <- function(summary_df, league){
                    #Shortening the season summary for a better ID
                    short_year = paste0(substr(summary$Season, 3, 4), "/", substr(summary$Season, 8, 9)) #e.g. 2021/2022 -> 21/22
                    league_initials = league_intitials(league) #e.g Converts Premier League to PL
                    
                    #Season, Team and Player ID
                    summary_df %<>%
                        mutate(Team_ID = as.character(glue("{short_year}-{Champions}"))) %>%
                        mutate(Player_ID = as.character(glue("{short_year}-{Top_Goal_Scorer}"))) 
        
    return(summary_df)
}

#Order columns
order_df = function(df){
            df = df %>%
                    select("Season_ID", "Season", "League", "Team_ID", "Champions", "Games_Won", 
                           "Player_ID", "Top_Goal_Scorer", "Goals_Scored")
    return(df)
}

### Main Program

In [3]:
#Preset varaibles
base_url = "https://fbref.com/en/comps/"             #Base URL
leagues = c("La Liga", "Premier League")             #Leagues we want to scrape information of
years = generate_years(2015, 2022)                   #Years we want to search for (start year, end year)
summary = tibble()                                   #Empty dataframe to load data into

#Main program
for (year in years){
    Sys.sleep(5)                                      #Delay scraping to not overload website
    for(league in leagues){
        link = season_html(year, base_url, league)
        champs = find_champs(link)
        wins = champs_wins(link, year, league)
        goals = top_goals(link)
        summary_df = update_summary(year, league, champs, wins, goals)
        summary_df = create_ids(summary_df, league)
        summary = order_df(summary_df)
    }
}


In [5]:
summary %>% select(c(Season, League, Champions, Games_Won, Top_Goal_Scorer, Goals_Scored)) %>% tail()

Season,League,Champions,Games_Won,Top_Goal_Scorer,Goals_Scored
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>
2019-2020,La Liga,Real Madrid,26,Lionel Messi,25
2019-2020,Premier League,Liverpool,32,Jamie Vardy,23
2020-2021,La Liga,Atlético Madrid,26,Lionel Messi,30
2020-2021,Premier League,Manchester City,27,Harry Kane,23
2021-2022,La Liga,Real Madrid,26,Karim Benzema,27
2021-2022,Premier League,Manchester City,29,Son Heung-min,23


In [4]:
#View of data
head(summary, 10)

Season_ID,Season,League,Team_ID,Champions,Games_Won,Player_ID,Top_Goal_Scorer,Goals_Scored
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>
89/90-LL,1989-1990,La Liga,89/90-Real Madrid,Real Madrid,26,89/90-Hugo Sánchez,Hugo Sánchez,38
89/90-PL,1989-1990,Premier League,89/90-Liverpool,Liverpool,23,89/90-NA,NA,NA
90/91-LL,1990-1991,La Liga,90/91-Barcelona,Barcelona,25,90/91-Emilio Butragueño,Emilio Butragueño,19
90/91-PL,1990-1991,Premier League,90/91-Arsenal,Arsenal,24,90/91-NA,NA,NA
91/92-LL,1991-1992,La Liga,91/92-Barcelona,Barcelona,23,91/92-Manolo,Manolo,27
91/92-PL,1991-1992,Premier League,91/92-Leeds United,Leeds United,22,91/92-NA,NA,NA
92/93-LL,1992-1993,La Liga,92/93-Barcelona,Barcelona,25,92/93-Bebeto,Bebeto,29
92/93-PL,1992-1993,Premier League,92/93-Manchester Utd,Manchester Utd,24,92/93-Teddy Sheringham,Teddy Sheringham,22
93/94-LL,1993-1994,La Liga,93/94-Barcelona,Barcelona,25,93/94-Romário,Romário,30


In [5]:
#Saving to csv file - commented out
#write.csv(summary, "C:/Users/guspa/Documents/Uni/Data 422 - Data Wrangling/Assignments/Group project/summary.csv")

In [10]:
#Skim summary
skim(summary)

── Data Summary ────────────────────────
                           Values 
Name                       summary
Number of rows             16     
Number of columns          9      
_______________________           
Column type frequency:            
  character                7      
  numeric                  2      
________________________          
Group variables            None   

── Variable type: character ────────────────────────────────────────────────────
  skim_variable   n_missing complete_rate min max empty n_unique whitespace
1 Season_ID               0             1   8   8     0       16          0
2 Season                  0             1   9   9     0        8          0
3 League                  0             1   7  14     0        2          0
4 Team_ID                 0             1  13  21     0       16          0
5 Champions               0             1   7  15     0        7          0
6 Player_ID               0             1  16  31     0       16       

ERROR: Error in is.null(text_repr) || nchar(text_repr) == 0L: 'length = 17' in coercion to 'logical(1)'


## Premier League - End of Year Tables

This contains all end of season tables for the Premier League dating back to 2015. Included are the number of matches played, matches won, matches lossed and matches drawn for each team - as well as a number of other statistics relating to overall team performance. Returned is a 160 x 25 dataset. <br> 

**Source: www.fbref.com** <br> <br>

**Note:** Statistics for expected goals, expected goals allowed, etc., were only recorded as far back as 2018 and are missing for earlier years. However, our analyses were not targeted at these variables. 

### Functions

In [11]:
#Retives end of year table for each year selected
get_tables = function(link, year, league){
                season = search_season(year) #Transforming year into search character and Season value
                node = glue("#results{season}91_overall") #Node to search for
    
                #Retrieving table
                season_table = link %>%        
                                html_nodes(node) %>%
                                html_table()  %>%
                                data.frame() %>%
                                tibble()
                
                #Adding Season + League column
                season_table %<>% 
                                mutate(Season = season,
                                League = league) 
                
                #Turning Attendance into numeric
                season_table$Attendance <- as.numeric(gsub(",", "", season_table$Attendance))
                
                #Updating tables dataframe
                if(is.null(tablesPL)){
                    tablesPL <<- season_table
                } else {
                    tablesPL <<- bind_rows(tablesPL, season_table)
                }
    
    return(tablesPL)
}


#Cleaning function to create and order df
table_cleaning = function(df){
                    #Split top goal scorer and goals scored into two separate columns
                    df<- df %>%
                          mutate(GoalsScored = as.numeric(sub(".* - ", "", Top.Team.Scorer )),
                                 Top.Team.Scorer = sub(" - .*", "", Top.Team.Scorer))    
    
                    #Shortening the season summary for a better ID (e.g. 2022-2023 -> 22/23)
                    short_year = paste0(substr(df$Season, 3, 4), "/", substr(df$Season, 8, 9))
    
                    #Season, Team and Player ID
                    df %<>%
                        mutate(Season_ID = as.character(glue("{short_year}-PL"))) %>%
                        mutate(Team_ID = as.character(glue("{short_year}-{Squad}"))) %>%
                        mutate(Player_ID = as.character(glue("{short_year}-{Top.Team.Scorer}"))) 
    
                    #Sorting order of columns
                    df = df %>%
                            select(Season_ID, Season, League, everything())
                    df = df %>%
                            relocate(Player_ID, .before = Top.Team.Scorer) %>%
                            relocate(Team_ID, .before = Squad) %>%
                            relocate(GoalsScored, .before = Goalkeeper)

    return(df)
}




### Main Program

In [12]:
#Preset varaibles
base_url = "https://fbref.com/en/comps/"            #Base URL
league = "Premier League"                           #League we want to scrape information of
years = generate_years(2015, 2022)                  #Years we want to search for (start year, end year)
tablesPL = tibble()                                 #Empty dataframe to load data into

#Scraping season tables
for (year in years){
        Sys.sleep(5)
        link = season_html(year, base_url, league)
        get_tables(link, year, league)    
}

#Tidying up collected data
tablesPL = table_cleaning(tablesPL)

In [15]:
#View of data
tail(tablesPL)

Season_ID,Season,League,Rk,Team_ID,Squad,MP,W,D,L,⋯,Attendance,Player_ID,Top.Team.Scorer,GoalsScored,Goalkeeper,Notes,xG,xGA,xGD,xGD.90
<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<int>,<int>,<int>,⋯,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
21/22-PL,2021-2022,Premier League,15,21/22-Southampton,Southampton,38,9,13,16,⋯,30134,21/22-James Ward-Prowse,James Ward-Prowse,10,Fraser Forster,,44.8,60.0,-15.2,-0.40
21/22-PL,2021-2022,Premier League,16,21/22-Everton,Everton,38,11,6,21,⋯,38446,21/22-Richarlison,Richarlison,10,Jordan Pickford,,41.2,55.4,-14.2,-0.37
21/22-PL,2021-2022,Premier League,17,21/22-Leeds United,Leeds United,38,9,11,18,⋯,36308,21/22-Raphinha,Raphinha,11,Illan Meslier,,44.4,67.8,-23.5,-0.62
21/22-PL,2021-2022,Premier League,18,21/22-Burnley,Burnley,38,7,14,17,⋯,19278,21/22-Maxwel Cornet,Maxwel Cornet,9,Nick Pope,Relegated,39.7,57.1,-17.4,-0.46
21/22-PL,2021-2022,Premier League,19,21/22-Watford,Watford,38,6,5,27,⋯,20614,21/22-Emmanuel Dennis,Emmanuel Dennis,10,Ben Foster,Relegated,40.0,64.5,-24.5,-0.64
21/22-PL,2021-2022,Premier League,20,21/22-Norwich City,Norwich City,38,5,7,26,⋯,26836,21/22-Teemu Pukki,Teemu Pukki,11,Tim Krul,Relegated,32.3,75.5,-43.2,-1.14


In [10]:
#Saving to csv file - commented out
#write.csv(tablesPL, "C:/Users/guspa/Documents/Uni/Data 422 - Data Wrangling/Assignments/Group project/PremierLeagueTables.csv")

In [14]:
#Skim summary
skim(tablesPL)

── Data Summary ────────────────────────
                           Values  
Name                       tablesPL
Number of rows             160     
Number of columns          25      
_______________________            
Column type frequency:             
  character                9       
  numeric                  16      
________________________           
Group variables            None    

── Variable type: character ────────────────────────────────────────────────────
  skim_variable   n_missing complete_rate min max empty n_unique whitespace
1 Season_ID               0             1   8   8     0        8          0
2 Season                  0             1   9   9     0        8          0
3 League                  0             1  14  14     0        1          0
4 Team_ID                 0             1   9  21     0      160          0
5 Squad                   0             1   3  15     0       32          0
6 Player_ID               0             1  14  40     0      

ERROR: Error in is.null(text_repr) || nchar(text_repr) == 0L: 'length = 17' in coercion to 'logical(1)'


## La Liga - End of Year Tables

Similarly to above, this dataset contains the end of year tables for the La Liga competition. Again, a 160 x 25 dataset is returned.<br>

**Source: www.fbref.com** 

### Functions

In [16]:
#Retives end of year table for each year selected
get_tablesLL = function(link, year, league){
                    season = search_season(year) #Transforming year into search character and Season value
                    node = glue("#results{season}121_overall") #Node to search for
    
                    #Retrieving table
                    season_table = link %>%        
                                    html_nodes(node) %>%
                                    html_table()  %>%
                                    data.frame() %>%
                                    tibble()
    
                    #Adding Season + League column
                    season_table %<>% 
                                    mutate(Season = season,
                                    League = league) 
    
                    #Turning Attendance into numeric
                    season_table$Attendance <- as.numeric(gsub(",", "", season_table$Attendance))
    
                    #Updating tables dataframe
                    if(is.null(tablesLL)){
                        tablesLL <<- season_table
                    } else {
                        tablesLL <<- bind_rows(tablesLL, season_table)
                    }
    
    return(tablesLL)
}


#Cleaning function to create and order df
table_cleaningLL = function(df){
                        #Split top goal scorer and goals scored into two separate columns
                        df<- df %>%
                              mutate(GoalsScored = as.numeric(sub(".* - ", "", Top.Team.Scorer )),
                                     Top.Team.Scorer = sub(" - .*", "", Top.Team.Scorer))   
    
                        #Shortening the season summary for a better ID (e.g. 2022-2023 -> 22/23)
                        short_year = paste0(substr(df$Season, 3, 4), "/", substr(df$Season, 8, 9))
    
                        #Season, Team and Player ID
                        df %<>%
                            mutate(Season_ID = as.character(glue("{short_year}-LL"))) %>%
                            mutate(Team_ID = as.character(glue("{short_year}-{Squad}"))) %>%
                            mutate(Player_ID = as.character(glue("{short_year}-{Top.Team.Scorer}"))) 
    
                        #Sorting order of columns
                        df = df %>%
                                select(Season_ID, Season, League, everything())
                        df = df %>%
                                relocate(Player_ID, .before = Top.Team.Scorer) %>%
                                relocate(Team_ID, .before = Squad) %>%
                                relocate(GoalsScored, .before = Goalkeeper)       

    return(df)
}

### Main Program

In [17]:
#Preset varaibles
base_url = "https://fbref.com/en/comps/"            #Base URL
league = "La Liga"                                  #League we want to scrape information of
years = generate_years(2015, 2022)                  #Years we want to search for (start year, end year)
tablesLL = tibble()                                 #Empty dataframe to load data into

#Scraping season tables
for (year in years){
            Sys.sleep(5)
            link = season_html(year, base_url, league)
            get_tablesLL(link, year, league)    
            }

#Tidying up collected data
tablesLL = table_cleaningLL(tablesLL)

In [18]:
#View of data
head(tablesLL)

Season_ID,Season,League,Rk,Team_ID,Squad,MP,W,D,L,⋯,Attendance,Player_ID,Top.Team.Scorer,GoalsScored,Goalkeeper,Notes,xG,xGA,xGD,xGD.90
<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<int>,<int>,<int>,⋯,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
14/15-LL,2014-2015,La Liga,1,14/15-Barcelona,Barcelona,38,30,4,4,⋯,77632,14/15-Lionel Messi,Lionel Messi,43,Claudio Bravo,→ Champions League via Champions League win,NA,NA,NA,NA
14/15-LL,2014-2015,La Liga,2,14/15-Real Madrid,Real Madrid,38,30,2,6,⋯,73918,14/15-Cristiano Ronaldo,Cristiano Ronaldo,48,Iker Casillas,→ Champions League via league finish,NA,NA,NA,NA
14/15-LL,2014-2015,La Liga,3,14/15-Atlético Madrid,Atlético Madrid,38,23,9,6,⋯,46532,14/15-Antoine Griezmann,Antoine Griezmann,22,Miguel Ángel Moyá,→ Champions League via league finish,NA,NA,NA,NA
14/15-LL,2014-2015,La Liga,4,14/15-Valencia,Valencia,38,22,11,5,⋯,44239,14/15-Daniel Parejo,Daniel Parejo,12,Diego Alves,→ Champions League via league finish,NA,NA,NA,NA
14/15-LL,2014-2015,La Liga,5,14/15-Sevilla,Sevilla,38,23,7,8,⋯,30874,14/15-Carlos Bacca,Carlos Bacca,20,Sergio Rico,→ Champions League via Europa League win,NA,NA,NA,NA
14/15-LL,2014-2015,La Liga,6,14/15-Villarreal,Villarreal,38,16,12,10,⋯,15660,14/15-Luciano Vietto,Luciano Vietto,12,Sergio Asenjo,→ Europa League via league finish 1,NA,NA,NA,NA


In [15]:
#Saving to csv - commented out
#write.csv(tablesLL, "C:/Users/guspa/Documents/Uni/Data 422 - Data Wrangling/Assignments/Group project/LaLigaTables.csv")

In [19]:
#Skim summary
skim(tablesLL)

── Data Summary ────────────────────────
                           Values  
Name                       tablesLL
Number of rows             160     
Number of columns          25      
_______________________            
Column type frequency:             
  character                9       
  numeric                  16      
________________________           
Group variables            None    

── Variable type: character ────────────────────────────────────────────────────
  skim_variable   n_missing complete_rate min max empty n_unique whitespace
1 Season_ID               0             1   8   8     0        8          0
2 Season                  0             1   9   9     0        8          0
3 League                  0             1   7   7     0        1          0
4 Team_ID                 0             1  11  21     0      160          0
5 Squad                   0             1   5  15     0       31          0
6 Player_ID               0             1  10  39     0      

ERROR: Error in is.null(text_repr) || nchar(text_repr) == 0L: 'length = 17' in coercion to 'logical(1)'


## Premier League - Fantasy Football

This dataset holds data pertaining to individual statistics for each player in the premier league. As we are now looking at individuals, as opposed to teams, this dataset is much larger than those previously. In this instance a 18,207 x 23 dataframe is returned.<br>

**Source: www.statbunker.com** <br><br>

**Note:** The statbunker website was found to very flaky. Often we found the website had crashed for multiple days at a time. If you plan on running the code - please click on the link above to ensure the website is up and running beforehand. 

### Functions

In [21]:
#Reads in base link to gain access to options of other links
fantasy_html <- function(url){      
                #Reading in link
                fantasy_html <- read_html(url)
    
    return(fantasy_html)
}


#Goes through the base html doc and collects ids for each season we want to investigate
season_links <- function(base_html){
                    links = list() #Empty list to add links to 
                    info = list() #Empty list to add league and year info
                    #Collects the comp name and id    
                    options = base_html %>%
                                html_element("select[name='comp_id']") %>%
                                html_nodes("option")
    
                    #Splits the options into its two components
                    for(option in options){
                        value <- option %>% html_attr("value") #Comp_id value - used to create link            
                        text <- option %>% html_text() #Year and League information
        
                        
                    #Base_url to add ids onto
                    base_url = "https://www.statbunker.com/competitions/FantasyFootballPlayersStats?"
                    
                    if (value != "-1" & value != "745" ) { #Don't want 2023/2024 season or select season options
                        new_url <- glue(base_url, "comp_id={value}")  #Create new url              
                        links <- append(links, new_url) #Add to list
                        info <- append(info, text) #Add to list
                        }
                        }
    return(list("links" = links, "info" = info)) #Return both links and info regarding link
    }
 

#Converts shortened years into full years - e.g. 22/23 -> 2022/2023
convert_full_years <- function(year){
    parts <- str_split(year, "/")[[1]]
    if(parts[1] <= 23){
        result = paste0("20", as.character(parts[1]), "-", "20", as.character(parts[2]))
    } else if (parts[1] == 99){
        result = paste0("19", as.character(parts[1]), "-", "20", as.character(parts[2]))
    }else{
        result = paste0("19", as.character(parts[1]), "-", "19", as.character(parts[2]))
    }
    return(result)
}


#Retrives all tables from fantasy football site
fantasy_table = function(links_and_info){
                for(i in seq_along(links_and_info$links)){                        #Run through each link
                    Sys.sleep(5)                                   #Sleep the system to not overload website
                    html_doc = read_html(as.character(links_and_info$links[i]))   #Read in the link
                    
                    #Select table and convert to tibble
                    season_table = html_doc %>%
                                    html_nodes("#mainContent > div > div > table") %>% #Selector
                                    html_table() %>%
                                    data.frame() %>%
                                    tibble()                    
                    
                    #Collecting year and league info
                    info = links_and_info$info[i]                             #Collect tag information 
                    info = strsplit(as.character(info), split = " ")[[1]]     #Split it up
                    league = paste(as.character(info[1:2]), collapse = " " )  #Select league info (Premier League)
                    year = as.character(info[3])                              #Select year (eg. 22/23)
                    long_year = convert_full_years(year)                      #Conver to long year (e.g. 2022-2023)
                    
                    #Create season_id, season and league columns
                    season_table %<>%
                                mutate(Season_ID = as.character(glue("{year}-PL")),
                                       Season = long_year,
                                       League = league)
                    
                    #Updating fantasyPL dataframe
                    if(is.null(fantasyPL)){
                        fantasyPL <<- season_table
                    } else {
                        fantasyPL <<- bind_rows(fantasyPL, season_table)
                    }    
                    }
    }


#Reorder and clean up fantasy dataframe
clean_fantasy <- function(df){
    df = subset(df, select = -More) #Remove More column
    
    #Adding Player ID
    short_year = substr(df$Season_ID, 1, 5) #Select shortened year for ID
    df %<>%
            mutate(Player_ID = as.character(glue("{short_year}-{Players}")))
    
    #Reordering df
    df = df %>%
            select(Season_ID, Season, League, Player_ID, everything())
    
  return(df)
}
    

### Main program

In [22]:
#Preset varaibles
url = "https://www.statbunker.com/competitions/FantasyFootballPlayersStats?comp_id=718" #Starting link
fantasyPL = tibble() #Empty df to load data into

#Main Program
base_html = fantasy_html(url)
links_and_info = season_links(base_html)
fantasy_table(links_and_info)

#Cleaning df
fantasyPL = clean_fantasy(fantasyPL)

Warning message in table_fill(cells, trim = trim):
"NAs introduced by coercion"


In [23]:
#View of data
head(fantasyPL)

Season_ID,Season,League,Player_ID,Players,Points,Clubs,Position,Start,Goals,⋯,Yellow,Red,Sub,CO,Off,Pen.SV,Pen.M,Goals.conceded,Conceded.1.,OG
<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<chr>,<int>,<chr>,<chr>,<int>
22/23-PL,2022-2023,Premier League,22/23-Erling Haaland,Erling Haaland,267,MCFC,Forward,33,36,⋯,5,0,3,2,12,-,0,-,-,0
22/23-PL,2022-2023,Premier League,22/23-Harry Kane,Harry Kane,227,SPURS,Forward,38,30,⋯,6,0,0,0,3,-,1,-,-,0
22/23-PL,2022-2023,Premier League,22/23-Mohamed Salah,Mohamed Salah,200,LPOOL,Forward,37,19,⋯,2,0,1,1,8,-,2,-,-,0
22/23-PL,2022-2023,Premier League,22/23-Bukayo Saka,Bukayo Saka,170,ARSL,Midfielder,37,14,⋯,6,0,1,1,14,-,1,-,-,0
22/23-PL,2022-2023,Premier League,22/23-Ivan Toney,Ivan Toney,167,BREN,Forward,33,20,⋯,9,0,0,0,4,-,1,-,-,0
22/23-PL,2022-2023,Premier League,22/23-Martin Odegaard,Martin Odegaard,166,ARSL,Midfielder,37,15,⋯,4,0,0,0,23,-,0,-,-,0


In [24]:
#Saving to csv - commented out
#write.csv(fantasyPL, "C:/Users/guspa/Documents/Uni/Data 422 - Data Wrangling/Assignments/Group project/PremierLeagueFantasy.csv")

In [26]:
#Skim summary
skim(fantasyPL)

── Data Summary ────────────────────────
                           Values   
Name                       fantasyPL
Number of rows             18207    
Number of columns          23       
_______________________             
Column type frequency:              
  character                12       
  numeric                  11       
________________________            
Group variables            None     

── Variable type: character ────────────────────────────────────────────────────
   skim_variable  n_missing complete_rate min max empty n_unique whitespace
 1 Season_ID              0          1      8   8     0       31          0
 2 Season                 0          1      9   9     0       31          0
 3 League                 0          1     14  14     0        1          0
 4 Player_ID              0          1      8  32     0    18187          0
 5 Players                0          1      2  26     0     5418          0
 6 Clubs                  1          1.00   3   7  

ERROR: Error in is.null(text_repr) || nchar(text_repr) == 0L: 'length = 17' in coercion to 'logical(1)'


## La Liga - Fantasy Football

This contains player data for players from the La Liga competition. Fantasy points do not date back as far for this competition so this dataset is slightly smaller - a 10,003 x 23 datset is returned.<br>

**Source: www.statbunker.com** <br><br>


### Functions

In [27]:
#Goes through the base html doc and collects ids for each season we want to investigate
season_linksLL <- function(base_html){
                    links = list() #Empty to list to add links to 
                    info = list()
                    #Collects the comp name and id    
                    options = base_html %>%
                                html_element("select[name='comp_id']") %>%
                                html_nodes("option")
    
                    #Splits the options into its two components
                    for(option in options){
                        value <- option %>% html_attr("value") #Comp_id value - used to create link            
                        text <- option %>% html_text() #Year and League information
        
                        
                    #Base_url to add ids onto
                    base_url = "https://www.statbunker.com/competitions/FantasyFootballPlayersStats?"
                    
                    if (value != "-1" & value != "753" ) { #Don't want 2023/2024 season or select season options
                        new_url <- glue(base_url, "comp_id={value}")  #Create new url              
                        links <- append(links, new_url) #Add to list
                        info <- append(info, text) #Add to list
                        }
                        }
    return(list("links" = links, "info" = info))
    }

#Retrives all tables from fantasy football site
fantasy_tableLL = function(links_and_info){
                for(i in seq_along(links_and_info$links)){                        #Run through each link
                    Sys.sleep(5)                                   #Sleep the system to not overload website
                    html_doc = read_html(as.character(links_and_info$links[i]))   #Read in the link
                    
                    #Select table and convert to tibble
                    season_table = html_doc %>%
                                    html_nodes("#mainContent > div > div > table") %>% #Selector
                                    html_table() %>%
                                    data.frame() %>%
                                    tibble()   
                    
                    
                    #Collecting year and league info
                    info = links_and_info$info[i]                          #Collect tag information 
                    info = strsplit(as.character(info), split = " ")[[1]]  #Split it up
                    league = paste(as.character(info[1:2]), collapse = " " )  #Select league info (Premier League)
                    year = as.character(info[3])                              #Select year (eg. 22/23)
                    long_year = convert_full_years(year)                      #Conver to long year (e.g. 2022-2023)
                    
                    #Create season_id, season and league columns
                    season_table %<>%
                                mutate(Season_ID = as.character(glue("{year}-LL")),
                                      Season = long_year,
                                        League = league)
                    
                    #Updating fantasyPL dataframe
                    if(is.null(fantasyLL)){
                        fantasyLL <<- season_table
                    } else {
                        fantasyLL <<- bind_rows(fantasyLL, season_table)
                    }    
                    }
    }





### Main program

In [28]:
#Preset varaibles
url = "https://www.statbunker.com/competitions/FantasyFootballPlayersStats?comp_id=730"
fantasyLL = tibble()

#Main program
initial_html = fantasy_html(url)
results = season_linksLL(initial_html)
fantasy_tableLL(results)

#Cleaning df
fantasyLL = clean_fantasy(fantasyLL)

In [30]:
#View of data
head(fantasyLL)

Season_ID,Season,League,Player_ID,Players,Points,Clubs,Position,Start,Goals,⋯,Yellow,Red,Sub,CO,Off,Pen.SV,Pen.M,Goals.conceded,Conceded.1.,OG
<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<chr>,<int>,<chr>,<chr>,<int>
22/23-LL,2022-2023,La Liga,22/23-Robert Lewandowski,Robert Lewandowski,198,BARC,Forward,33,23,⋯,3,0,1,1,5,-,1,-,-,0
22/23-LL,2022-2023,La Liga,22/23-Antoine Griezmann,Antoine Griezmann,190,,Forward,31,15,⋯,2,0,7,7,8,-,0,-,-,0
22/23-LL,2022-2023,La Liga,22/23-Marc-Andre ter Stegen,Marc-Andre ter Stegen,172,BARC,Goalkeeper,38,0,⋯,0,0,0,0,2,0,0,20,8,0
22/23-LL,2022-2023,La Liga,22/23-Borja Iglesias,Borja Iglesias,158,BETI,Defender,29,15,⋯,2,1,6,6,22,-,1,33,11,0
22/23-LL,2022-2023,La Liga,22/23-Joselu,Joselu,151,RCDE,Forward,33,16,⋯,2,0,2,1,0,-,0,-,-,0
22/23-LL,2022-2023,La Liga,22/23-Alejandro Balde,Alejandro Balde,149,BARC,Defender,30,1,⋯,4,0,7,3,9,-,0,17,7,0


In [31]:
#Saving to csv - commented out
#write.csv(fantasyLL, "C:/Users/guspa/Documents/Uni/Data 422 - Data Wrangling/Assignments/Group project/LaLigaFantasy.csv")

In [32]:
#Skim summary
skim(fantasyLL)

── Data Summary ────────────────────────
                           Values   
Name                       fantasyLL
Number of rows             10003    
Number of columns          23       
_______________________             
Column type frequency:              
  character                12       
  numeric                  11       
________________________            
Group variables            None     

── Variable type: character ────────────────────────────────────────────────────
   skim_variable  n_missing complete_rate min max empty n_unique whitespace
 1 Season_ID              0             1   8   8     0       16          0
 2 Season                 0             1   9   9     0       16          0
 3 League                 0             1   7   7     0        1          0
 4 Player_ID              0             1   9  32     0     9898          0
 5 Players                0             1   3  26     0     3357          0
 6 Clubs                  0             1   0   4  

ERROR: Error in is.null(text_repr) || nchar(text_repr) == 0L: 'length = 17' in coercion to 'logical(1)'


## Premier League - Wages

This dataset contains the weekly and annual wages spent by each team in the Premier League on a given season. A 160 x 10 dataframe is returned. <br>

**Source: www.fbref.com** 

### Functions

In [34]:
#Converts League into format of website
convert_league_wages = function(league){
                    words = unlist(strsplit(league, " "))
                    conversion = glue("-", words[1], "-", words[2], "-Wages")
    
    return(conversion)    
}

#Creates link for initial seach of each year 
season_html_wages <- function(year, base_url, league){
                #Collecting pieces to create link
                season = search_season(year)
                league_conversion = convert_league_wages(league)
    
                #Creating link
                link = glue(base_url, season, "/wages/", season, league_conversion)
    
                #Reading in link
                season_html <- read_html(link)
    
    return(season_html)
}


get_wages_tables = function(link, year, league){
                season = search_season(year) #Transforming year into search character and Season value
    
                #Retrieving table
                season_table = link %>%        
                                html_nodes("#div_squad_wages") %>%
                                html_table()  %>%
                                data.frame() %>%
                                tibble()
    
                #Adding Season + League column
                season_table %<>% 
                                mutate(Season = season,
                                League = league) 
    
                #Updating tables dataframe
                if(is.null(wagesPL)){
                    wagesPL <<- season_table
                } else {
                    wagesPL <<- bind_rows(wagesPL, season_table)
                }
    return(wagesPL)
    }


#Create ID Columns
wages_ids <- function(df, league){
                    #Shortening the season summary for a better ID
                    short_year = paste0(substr(df$Season, 3, 4), "/", substr(df$Season, 8, 9)) #e.g. 2021/2022 -> 21/22
                    league_initials = league_intitials(league) #e.g Converts Premier League to PL
                    
                    #Season, Team and Player ID
                    df %<>%
                        mutate(Team_ID = as.character(glue("{short_year}-{Squad}"))) %>%
                        mutate(Season_ID = as.character(glue("{short_year}-{league_initials}"))) 
        
    return(df)
}

order_wages <- function(df){
    df %<>%
        select(Season_ID, Season, League, everything())
    df %<>%
        relocate(Team_ID, .before = Squad)
    
    
    return(df)
}

convert_to_pounds <- function(df){
    df %<>% 
        mutate(Annual_Wages_pounds = as.numeric(gsub("\\([^\\)]*\\)|[£,]", "", df$Annual.Wages)),
               Weekly_Wages_pounds = as.numeric(gsub("\\([^\\)]*\\)|[£,]", "", df$Weekly.Wages)))
    
    df %<>%
        select(-c(Annual.Wages, Weekly.Wages))
    
    df%<>%
        relocate(Weekly_Wages_pounds, Annual_Wages_pounds, .before = X..Estimated )
    
    return(df)
}

### Main program

In [35]:
#Preset varaibles
base_url = "https://fbref.com/en/comps/9/" #Base url
league = "Premier League"                  #League we wish to investigate
years = generate_years(2015, 2022)         #Year selection
wagesPL = tibble()                         #Empty df to add data to

#Collect data
for(year in years){
    Sys.sleep(5)
            read_link = season_html_wages(year, base_url, league)
             table = get_wages_tables(read_link, year, league)
    }

#Data processing
wagesPL %<>%
    wages_ids(league) %>%
    order_wages() %>%
    convert_to_pounds()

In [36]:
#View of data
head(wagesPL)

Season_ID,Season,League,Rk,Team_ID,Squad,X..Pl,Weekly_Wages_pounds,Annual_Wages_pounds,X..Estimated
<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<chr>
14/15-PL,2014-2015,Premier League,1,14/15-Manchester Utd,Manchester Utd,43,2713846,141120000,100%
14/15-PL,2014-2015,Premier League,2,14/15-Manchester City,Manchester City,36,2351846,122296000,100%
14/15-PL,2014-2015,Premier League,3,14/15-Chelsea,Chelsea,38,2012385,104644000,100%
14/15-PL,2014-2015,Premier League,4,14/15-Arsenal,Arsenal,42,1820712,94677000,100%
14/15-PL,2014-2015,Premier League,5,14/15-Liverpool,Liverpool,35,1577500,82030000,100%
14/15-PL,2014-2015,Premier League,6,14/15-Tottenham,Tottenham,43,1265192,65790000,100%


In [27]:
#Saving to csv - commented out
#write.csv(wagesPL, "C:/Users/guspa/Documents/Uni/Data 422 - Data Wrangling/Assignments/Group project/PremierLeagueWages.csv")

In [37]:
#Skim summary
skim(wagesPL)

── Data Summary ────────────────────────
                           Values 
Name                       wagesPL
Number of rows             160    
Number of columns          10     
_______________________           
Column type frequency:            
  character                6      
  numeric                  4      
________________________          
Group variables            None   

── Variable type: character ────────────────────────────────────────────────────
  skim_variable n_missing complete_rate min max empty n_unique whitespace
1 Season_ID             0             1   8   8     0        8          0
2 Season                0             1   9   9     0        8          0
3 League                0             1  14  14     0        1          0
4 Team_ID               0             1   9  21     0      160          0
5 Squad                 0             1   3  15     0       32          0
6 X..Estimated          0             1   2   4     0       33          0

── Varia

ERROR: Error in is.null(text_repr) || nchar(text_repr) == 0L: 'length = 17' in coercion to 'logical(1)'


## La liga - Wages

This dataset contains the weekly and annual wages spent by each team in the La Liga on a given season. A 157 x 10 dataframe is returned. Slightly fewer rows are found in this dataset as the La Liga competition only expanded to 20 teams in 2016.<br>

**Source: www.fbref.com** 

### Functions 

In [38]:
#Few minor adjustments to original get_tables for the PL
get_wages_tablesLL = function(link, year, league){
                season = search_season(year) #Transforming year into search character and Season value
    
                #Retrieving table
                season_table = link %>%        
                                html_nodes("#div_squad_wages") %>%
                                html_table()  %>%
                                data.frame() %>%
                                tibble()
    
                #Adding Season + League column
                season_table %<>% 
                                mutate(Season = season,
                                       League = league) 
    
                #Updating tables dataframe
                if(is.null(wagesLL)){
                    wagesLL <<- season_table
                } else {
                    wagesLL <<- bind_rows(wagesLL, season_table)
                }
    
    return(wagesLL)
    }

#Extract the pound value of team wages and does a bit of reshuffling 
convert_to_poundsLL <- function(df){
    #Creating columns with wages in pounds
    df %<>% 
        mutate(Annual_Wages_pounds = as.numeric(gsub(",", "", str_extract(wagesLL$Annual.Wages, "(?<=£ )\\d+(?:,\\d+)*"), fixed = TRUE)),
               Weekly_Wages_pounds = as.numeric(gsub(",", "", str_extract(wagesLL$Weekly.Wages, "(?<=£ )\\d+(?:,\\d+)*"), fixed = TRUE)))
    
    #Dropping redundant columns
    df %<>%
        select(-c(Annual.Wages, Weekly.Wages))
    
    #Reordering newly added columns
    df%<>%
        relocate(Weekly_Wages_pounds, Annual_Wages_pounds, .before = X..Estimated )
    
    return(df)
}

### Main program

In [39]:
#Preset varaibles
base_url = "https://fbref.com/en/comps/12/" #Base url
league = "La Liga"                          #League we wish to investigate
years = generate_years(2015, 2022)          #Year selection
wagesLL = tibble()                          #Empty df to add data to

#Collect data
for(year in years){
    Sys.sleep(5)
            read_link = season_html_wages(year, base_url, league)
             table = get_wages_tablesLL(read_link, year, league)
    }

#Data processing
wagesLL %<>%
    wages_ids(league) %>%
    order_wages() %>%
    convert_to_poundsLL()

In [40]:
#View of data
head(wagesLL)

Season_ID,Season,League,Rk,Team_ID,Squad,X..Pl,Weekly_Wages_pounds,Annual_Wages_pounds,X..Estimated
<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<chr>
14/15-LL,2014-2015,La Liga,1,14/15-Barcelona,Barcelona,31,2614969,135978388,100%
14/15-LL,2014-2015,La Liga,2,14/15-Real Madrid,Real Madrid,42,2473196,128606217,100%
14/15-LL,2014-2015,La Liga,3,14/15-Atlético Madrid,Atlético Madrid,32,954948,49657303,100%
14/15-LL,2014-2015,La Liga,4,14/15-Valencia,Valencia,26,488802,25417705,100%
14/15-LL,2014-2015,La Liga,5,14/15-Villarreal,Villarreal,37,434783,22608702,100%
14/15-LL,2014-2015,La Liga,6,14/15-Sevilla,Sevilla,35,423930,22044385,100%


In [41]:
#Saving to csv - commented out
#write.csv(wagesLL, "C:/Users/guspa/Documents/Uni/Data 422 - Data Wrangling/Assignments/Group project/LaLigaWages.csv")

In [42]:
#Skim summary
skim(wagesLL)

── Data Summary ────────────────────────
                           Values 
Name                       wagesLL
Number of rows             157    
Number of columns          10     
_______________________           
Column type frequency:            
  character                6      
  numeric                  4      
________________________          
Group variables            None   

── Variable type: character ────────────────────────────────────────────────────
  skim_variable n_missing complete_rate min max empty n_unique whitespace
1 Season_ID             0             1   8   8     0        8          0
2 Season                0             1   9   9     0        8          0
3 League                0             1   7   7     0        1          0
4 Team_ID               0             1  11  21     0      157          0
5 Squad                 0             1   5  15     0       31          0
6 X..Estimated          0             1   2   4     0       25          0

── Varia

ERROR: Error in is.null(text_repr) || nchar(text_repr) == 0L: 'length = 17' in coercion to 'logical(1)'


## Summary

All the datasets collected above are able to be merged together by either the Season_ID, Team_ID or Player_ID columns. These transform our separate datasets into a relational database. More details on this process and additional information regarding challenges faced can be found in our report. <br><br>
All datasets were saved as csv files, to allow other students in the group to access the data without having to execute any of the code. This not only saved time, but ensured that the data was readily accessible - not always a guarantee with data collected from statbunker.com.